In [1]:
pip install numpy==1.24 scipy==1.13 smart_open


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install gensim==4.3.3

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [22]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\William\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
#Import libraries
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
import nltk
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pickle
from sklearn.preprocessing import LabelEncoder

c:\Users\William\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Download NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
#Import datasets
twitter_train = pd.read_csv('raw data\\twitter_training.csv', header=None, names=['Tweet id','topic', 'sentiment','Tweet content'])
twitter_val = pd.read_csv('raw data\\twitter_validation.csv', header=None, names=['Tweet id','topic', 'sentiment','Tweet content'])
twitter = pd.concat([twitter_train, twitter_val], ignore_index=True)
emotion_text = pd.read_csv('raw data\\tweet_emotions.csv')
youtube_comments = pd.read_csv('raw data\\YoutubeCommentsDataSet.csv')

In [5]:
#Clean data (duplicates, missing values, uniform text, etc.)
def clean_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remove user mentions and hashtags
        text = re.sub(r'@\w+|#\w+', '', text)
        # Remove punctuation and special characters
        text = re.sub(r'[^\w\s]', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Remove stopwords and lemmatize
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
        
        return ' '.join(words)
    return ''

In [6]:
#Drop irrelevant rows
twitter = twitter[twitter['sentiment'] != 'Irrelevant']

In [7]:
#Map youtube labels
capital = {
    'positive': 'Positive',
    'negative': 'Negative',
    'neutral': 'Neutral'
}
youtube_comments['Sentiment'] = youtube_comments['Sentiment'].map(capital)

In [8]:
#Map emotions to uniform data
emotion_to_sentiment = {
    'enthusiasm': 'Positive',
    'surprise': 'Positive',
    'love': 'Positive',
    'fun': 'Positive',
    'happiness': 'Positive',
    'neutral': 'Neutral',
    'relief': 'Positive',
    'anger': 'Negative',
    'boredom': 'Negative',
    'hate': 'Negative',
    'worry': 'Negative',
    'sadness': 'Negative',
    'empty': 'Negative'
}

# Apply mapping to emotion dataset
emotion_text['sentiment'] = emotion_text['sentiment'].map(emotion_to_sentiment)

In [9]:
#Preprocess data
# Twitter dataset
twitter['cleaned_text'] = twitter['Tweet content'].apply(clean_text)
twitter_subset = twitter[['cleaned_text', 'sentiment']]

# YouTube dataset
youtube_comments['cleaned_text'] = youtube_comments['Comment'].apply(clean_text)
youtube_subset = youtube_comments[['cleaned_text', 'Sentiment']]
youtube_subset = youtube_subset.rename(columns={'Sentiment': 'sentiment'})

# Emotion dataset
emotion_text['cleaned_text'] = emotion_text['content'].apply(clean_text)
emotion_subset = emotion_text[['cleaned_text', 'sentiment']]

In [10]:
#Concat data into one dataset
combined_data = pd.concat([twitter_subset, youtube_subset, emotion_subset], ignore_index=True)
combined_data = combined_data.dropna()

In [11]:
#drop duplicates
combined_data = combined_data.drop_duplicates()

In [12]:
#Encode data
label_encoder = LabelEncoder()
combined_data['sentiment_encoded'] = label_encoder.fit_transform(combined_data['sentiment'])


In [13]:
# Print the mapping for reference
sentiment_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Sentiment mapping:", sentiment_mapping)

Sentiment mapping: {'Negative': 0, 'Neutral': 1, 'Positive': 2}


In [14]:
# Create Hybrid Tokenizer class for ECO-SAM, UFEN, and Neuro-Symbolic Transformer with LDA
class HybridTokenizer:
    def __init__(self, max_length=128, lda_num_topics=10):
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length
        self.lda_num_topics = lda_num_topics
        self.lda_dictionary = None
        self.lda_model = None

    def fit_lda(self, texts):
        # Tokenize texts for LDA
        tokenized_texts = [word_tokenize(text.lower()) for text in texts]
        self.lda_dictionary = Dictionary(tokenized_texts)
        corpus = [self.lda_dictionary.doc2bow(text) for text in tokenized_texts]
        self.lda_model = LdaModel(corpus=corpus, id2word=self.lda_dictionary, num_topics=self.lda_num_topics)

    def tokenize(self, text):
        # BERT tokenization for ECO-SAM
        bert_encoding = self.bert_tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Word tokenization for UFEN
        words = word_tokenize(text.lower())

        # LDA topic distribution for Neuro-Symbolic Transformer
        if self.lda_model is not None:
            bow = self.lda_dictionary.doc2bow(words)
            topic_dist = self.lda_model.get_document_topics(bow)
            # Convert sparse topic distribution to dense
            dense_topics = [0] * self.lda_num_topics
            for topic_id, prob in topic_dist:
                dense_topics[topic_id] = prob
            topic_dist = dense_topics
        else:
            topic_dist = [0] * self.lda_num_topics

        return {
            'input_ids': bert_encoding['input_ids'].squeeze(),
            'attention_mask': bert_encoding['attention_mask'].squeeze(),
            'words': words,
            'topic_dist': torch.tensor(topic_dist, dtype=torch.float)
        }

In [15]:
combined_data.head()

,cleaned_text,sentiment,sentiment_encoded
0,im getting borderland murder,Positive,2
1,coming border kill,Positive,2
2,im getting borderland kill,Positive,2
3,im coming borderland murder,Positive,2
6,spent hour making something fun dont know huge...,Positive,2


In [16]:
# Initialize the tokenizer
tokenizer = HybridTokenizer(max_length=128, lda_num_topics=10)

# Fit LDA model on the cleaned text data
print("Fitting LDA model...")
tokenizer.fit_lda(combined_data['cleaned_text'])

# Create train, validation, and test splits
from sklearn.model_selection import train_test_split

# Create stratified splits to maintain class distribution
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42, stratify=combined_data['sentiment_encoded'])
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42, stratify=train_data['sentiment_encoded'])

print(f"Train set: {len(train_data)} samples")
print(f"Validation set: {len(val_data)} samples")
print(f"Test set: {len(test_data)} samples")

# Function to tokenize and store results
def tokenize_and_store(text):
    tokens = tokenizer.tokenize(text)
    return {
        'input_ids': tokens['input_ids'].tolist(),
        'attention_mask': tokens['attention_mask'].tolist(),
        'topic_dist': tokens['topic_dist'].tolist()
    }

# Apply tokenization to each split
print("Tokenizing datasets...")
train_tokenized = train_data['cleaned_text'].apply(tokenize_and_store)
val_tokenized = val_data['cleaned_text'].apply(tokenize_and_store)
test_tokenized = test_data['cleaned_text'].apply(tokenize_and_store)

# Add tokenized data to each dataframe
train_data['input_ids'] = train_tokenized.apply(lambda x: x['input_ids'])
train_data['attention_mask'] = train_tokenized.apply(lambda x: x['attention_mask'])
train_data['topic_dist'] = train_tokenized.apply(lambda x: x['topic_dist'])

val_data['input_ids'] = val_tokenized.apply(lambda x: x['input_ids'])
val_data['attention_mask'] = val_tokenized.apply(lambda x: x['attention_mask'])
val_data['topic_dist'] = val_tokenized.apply(lambda x: x['topic_dist'])

test_data['input_ids'] = test_tokenized.apply(lambda x: x['input_ids'])
test_data['attention_mask'] = test_tokenized.apply(lambda x: x['attention_mask'])
test_data['topic_dist'] = test_tokenized.apply(lambda x: x['topic_dist'])

# Save the dataframes
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

# Save tokenized arrays for efficient model training
np.save('train_input_ids.npy', np.array(train_data['input_ids'].tolist()))
np.save('train_attention_mask.npy', np.array(train_data['attention_mask'].tolist()))
np.save('train_topic_dist.npy', np.array(train_data['topic_dist'].tolist()))
np.save('train_labels.npy', np.array(train_data['sentiment_encoded']))

np.save('val_input_ids.npy', np.array(val_data['input_ids'].tolist()))
np.save('val_attention_mask.npy', np.array(val_data['attention_mask'].tolist()))
np.save('val_topic_dist.npy', np.array(val_data['topic_dist'].tolist()))
np.save('val_labels.npy', np.array(val_data['sentiment_encoded']))

np.save('test_input_ids.npy', np.array(test_data['input_ids'].tolist()))
np.save('test_attention_mask.npy', np.array(test_data['attention_mask'].tolist()))
np.save('test_topic_dist.npy', np.array(test_data['topic_dist'].tolist()))
np.save('test_labels.npy', np.array(test_data['sentiment_encoded']))

# Save the tokenizer for future use
with open('hybrid_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Tokenization and data splitting completed successfully!")

Fitting LDA model...
Train set: 64563 samples
Validation set: 21522 samples
Test set: 21522 samples
Tokenizing datasets...
Tokenization and data splitting completed successfully!
